In [7]:
from ecg_batch import *
from dataset.dsindex import FilesIndex, DatasetIndex
import numpy as np
import matplotlib.pyplot as plt
import time

DIR_ECG = '/notebooks/data/ECG/training2017/'
files = np.array(open(DIR_ECG
                      + 'RECORDS', 'r').read().splitlines())[:10]

paths = [DIR_ECG + x for x in files]
ind = DatasetIndex(index = files)

src = dict(zip(files, paths))
ecg_batch = EcgBatch(ind.indices)

In [8]:
startTime = time.time()
ecg_batch = (ecg_batch.load(src)
             .gradient(order=1).gradient(order=2)
             .convolve('grad_1').convolve('grad_2')
             .wavelet_filter('db4', from_levels=np.array([3, 4]), t_low=90, t_high=95)
             .convolve('DWT_filter')
             .hmm_estimate(layers=['grad_2_conv', 'DWT_filter_conv'], 
                           n_components=4, n_iter=5)
             .get_r_peaks(prior='DWT_filter_conv')
             .dump('./dump', fmt='npz')
            )
print(time.time() - startTime)

3.2063143253326416


In [9]:
ds = Dataset(index=DatasetIndex(index = files), batch_class=EcgBatch)

startTime = time.time()
(ds.pipeline().load(src)
              .gradient(order=1).gradient(order=2)
              .convolve('grad_1').convolve('grad_2')
              .wavelet_filter('db4', from_levels=np.array([3, 4]), t_low=90, t_high=95)
              .convolve('DWT_filter')
              .hmm_estimate(layers=['grad_2_conv', 'DWT_filter_conv'], 
                            n_components=4, n_iter=5)
              .get_r_peaks(prior='DWT_filter_conv')
              .dump('./dump', fmt='npz')
              .run(batch_size=len(src)//2, shuffle=False, n_epochs=1, prefetch=2, target='mpc'))
print(time.time() - startTime)

2.0881316661834717
